In [ ]:
import cv2
import os
from PIL import Image
import numpy as np
dim = (30, 30)
X_train =[]
Y_train=[]
n=0

#read train data
path='/kaggle/input/gtsrb-german-traffic-sign/train'
grup = os.listdir(path)
for i in grup:
    tem=os.path.join(path, i)
    img_nams=os.listdir(tem)
    n+=1
    for imasg in img_nams:
        img=np.array(Image.open(os.path.join(tem,imasg )).resize((30,30),Image.ANTIALIAS))
        X_train.append(img)
        s=int(i)
        Y_train.append(s)



X_t=np.array(X_train)
X_t = X_t.astype('float32')
X_train=X_t/255.0


Y_t=np.array(Y_train)
from keras.utils import np_utils
Y_train=np_utils.to_categorical(Y_t)
#print(n)

#SHUFFLE DATA
from sklearn.utils import shuffle
X_train, Y_train = shuffle(X_train,Y_train, random_state=0)




In [ ]:
# relu model without dropout
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5) ,activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3),strides =2 , activation='relu'))
#model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3),strides =2, activation='relu'))
#model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
# model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


trained_model = model.fit(X_train, Y_train, batch_size=128, epochs=100 , validation_split = 0.2)

history=trained_model.history



In [ ]:
# relu model with dropout
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model1 = Sequential()
model1.add(Conv2D(filters=32, kernel_size=(5,5) ,activation='relu', input_shape=X_train.shape[1:]))
model1.add(Conv2D(filters=64, kernel_size=(3, 3),strides= 2 , activation='relu'))
#model.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(rate=0.25))
model1.add(Conv2D(filters=64, kernel_size=(3, 3),strides= 2, activation='relu'))
#model.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(rate=0.25))
model1.add(Flatten())
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(rate=0.5))
model1.add(Dense(43, activation='softmax'))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


trained_model1 = model1.fit(X_train, Y_train, batch_size=128, epochs=100 , validation_split = 0.2)

history1=trained_model1.history

In [ ]:
#val drop

import matplotlib.pyplot as plt

val_acc_with_dropout = history['val_accuracy']
train_acc_with_dropout=history['accuracy']

train_losses_with_dropout=history['loss']
val_losses_with_dropout=history['val_loss']




train_losses_without_dropout =history1['loss']
val_losses_without_dropout=history1['val_loss']

val_acc_without_dropout = history1['val_accuracy']
train_acc_without_dropout=history1['accuracy']

plt.figure(figsize=(5,5))
plt.xlabel('Epochs')
plt.ylabel('val_accuracy')
plt.plot(val_acc_with_dropout)
plt.plot(val_acc_without_dropout)
plt.legend(['with_dropout','without_dropout'])
plt.show()

plt.figure(figsize=(5,5))
plt.xlabel('Epochs')
plt.ylabel('train_accuracy')
plt.plot(train_acc_with_dropout)
plt.plot(train_acc_without_dropout)
plt.legend(['with_dropout','without_dropout'])
plt.show()



plt.figure(figsize=(5,5))
plt.xlabel('Epochs')
plt.ylabel('val_loss')
plt.plot(val_losses_with_dropout)
plt.plot(val_losses_without_dropout)
plt.legend(['with_dropout','without_dropout'])
plt.show()


plt.figure(figsize=(5,5))
plt.xlabel('Epochs')
plt.ylabel('train_loss')
plt.plot(train_losses_with_dropout)
plt.plot(train_losses_without_dropout)
plt.legend(['with_dropout','without_dropout'])
plt.show()


In [ ]:
from keras.utils import plot_model

plot_model(model ,to_file = 'model.png' , show_shapes=True, show_layer_names=True )


import matplotlib.pyplot as plt


losses=history['loss']
val_losses=history['val_loss']

acc=history['accuracy']
val_acc = history['val_accuracy']



plt.figure(figsize=(10,10))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(losses)
plt.plot(val_losses)
plt.legend(['train loss','val_loss'])

plt.figure(figsize=(10,10))
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.plot(acc)
plt.plot(val_acc)
plt.legend(['train accuracy','val_accuracy'])

In [ ]:
#read test data
import pandas as pd
data = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')

X_test =[]
Y_test=[]
path='/kaggle/input/gtsrb-german-traffic-sign/test'

grup = os.listdir(path)
for imasg in grup:
    n+=1
    if(imasg!='GT-final_test.csv'):
        img=np.array(Image.open(os.path.join(path,imasg )).resize((30,30),Image.ANTIALIAS))
        X_test.append(img)
        p=(os.path.join('Test',imasg ))
        Y_test.append(data.loc[data['Path'] == p, 'ClassId'].iloc[0])

        
        
X_t=np.array(X_test)
X_t = X_t.astype('float32')
X_test=X_t/255.0

print(X_test.shape)


Y_t=np.array(Y_test)
from keras.utils import np_utils
Y_test=np_utils.to_categorical(Y_t)
print(Y_test.shape)

test_loss,test_acc=model1.evaluate(X_test,Y_test)


predicted_labels=model1.predict(X_test)
print(predicted_labels.shape)
print("test_loss",test_loss)
print("test_acc" , test_acc)